In [16]:
# Please execute/shift-return this cell everytime you run the notebook.  Don't edit it. 
%load_ext autoreload
%autoreload 2
from notebook import * 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


 # Memory hierarchy

## How much time did I spent on "memory"

Remember our demo long time ago?

In [ ]:
compare([do_render_code("./madd/madd_A.c", lang="c++", show=["//START", "//END"]),do_render_code("./madd/madd_B.c", lang="c++", show=["//START", "//END"])])

In [17]:
render_code("./mv/matvec.c", lang="c++", show=["//I_FIRST_START", "//I_FIRST_END"])

// ./mv/matvec.c:35-49 (15 lines)
  //I_FIRST_START
    for(i = 0; i < ARRAY_SIZE; i++)
    {
      result = 0.0;    
      for(j = 0; j < ARRAY_SIZE; j++)
      {
        result += a[i][j]*b[j];
        if(dump)
            printf("&a[%d][%d], %p\n&b[%d], %p\n", i, j, &a[i][j], j, &b[j]);
      }
      c[i] = result;
      if(dump)
          printf("&c[%d], %p\n", i, &c[i]);
    }
  //I_FIRST_END

In [18]:
! cd mv; make clean; make; ./matvec 16 1 1 >> mv_addresses.csv ; head -n 100 mv_addresses.csv

rm -f matvec
cc -DHAVE_LINUX_PERF_EVENT_H -g -O3  matvec.c -o matvec
&a[0][0], 0x55ac54a78330
&b[0], 0x55ac54a78c30
&a[0][1], 0x55ac54a78338
&b[1], 0x55ac54a78c38
&a[0][2], 0x55ac54a78340
&b[2], 0x55ac54a78c40
&a[0][3], 0x55ac54a78348
&b[3], 0x55ac54a78c48
&a[0][4], 0x55ac54a78350
&b[4], 0x55ac54a78c50
&a[0][5], 0x55ac54a78358
&b[5], 0x55ac54a78c58
&a[0][6], 0x55ac54a78360
&b[6], 0x55ac54a78c60
&a[0][7], 0x55ac54a78368
&b[7], 0x55ac54a78c68
&a[0][8], 0x55ac54a78370
&b[8], 0x55ac54a78c70
&a[0][9], 0x55ac54a78378
&b[9], 0x55ac54a78c78
&a[0][10], 0x55ac54a78380
&b[10], 0x55ac54a78c80
&a[0][11], 0x55ac54a78388
&b[11], 0x55ac54a78c88
&a[0][12], 0x55ac54a78390
&b[12], 0x55ac54a78c90
&a[0][13], 0x55ac54a78398
&b[13], 0x55ac54a78c98
&a[0][14], 0x55ac54a783a0
&b[14], 0x55ac54a78ca0
&a[0][15], 0x55ac54a783a8
&b[15], 0x55ac54a78ca8
&c[0], 0x55ac54a78cc0
&a[1][0], 0x55ac54a783c0
&b[0], 0x55ac54a78c30
&a[1][1], 0x55ac54a783c8
&b[1], 0x55ac54a78c38
&a[1][2], 0x55ac54a783d0
&b[2], 0x55ac54a78c40
&a[1

In [ ]:
! cd mv; ./matvec 16 1 0 >> mv_addresses_j_first.csv ; head -n 100 mv_addresses_j_first.csv

## The memory hierarchy of my computer

In [15]:
# Hung-Wei's Desktop
! lscpu | grep "Model name"
! getconf -a | grep CACHE

Model name:                      13th Gen Intel(R) Core(TM) i7-13700
LEVEL1_ICACHE_SIZE                 32768
LEVEL1_ICACHE_ASSOC                8
LEVEL1_ICACHE_LINESIZE             64
LEVEL1_DCACHE_SIZE                 49152
LEVEL1_DCACHE_ASSOC                12
LEVEL1_DCACHE_LINESIZE             64
LEVEL2_CACHE_SIZE                  2097152
LEVEL2_CACHE_ASSOC                 16
LEVEL2_CACHE_LINESIZE              64
LEVEL3_CACHE_SIZE                  31457280
LEVEL3_CACHE_ASSOC                 12
LEVEL3_CACHE_LINESIZE              64
LEVEL4_CACHE_SIZE                  0
LEVEL4_CACHE_ASSOC                 0
LEVEL4_CACHE_LINESIZE              0


In [ ]:
# Your CS203 Cluster
! cse142 job run "lscpu | grep 'Model name'; getconf -a | grep CACHE"

In [ ]:
! ssh htseng@nano "lscpu | grep 'Model name'; getconf -a | grep CACHE"

## How blocks are stored in a cache?

How these address are stored "if" they're in a direct-mapped, 16B-sized blocks, 16-block cache?

In [ ]:
! cd mv; make clean; make; ./matvec 16 1 

In [ ]:
!echo "element,address"> addresses.csv; ./mv/matvec 16 1 >> addresses.csv
df = pd.read_csv("addresses.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
block_size = 16
offset_bits = int(math.log2(block_size))
number_of_blocks = 16
index_bits = int(math.log2(number_of_blocks))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % number_of_blocks))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

What if we have a 2-way, 16-byte blocked, 16-block cache?

In [ ]:
df = pd.read_csv("addresses.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
block_size = 16
offset_bits = int(math.log2(block_size))
number_of_blocks = 16
way_assoc=2
number_of_sets = number_of_blocks/way_assoc
index_bits = int(math.log2(number_of_sets))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % number_of_blocks))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

## Cache performance of code on "real machines"

### NVIDIA Jetson Nano -- Tegra X1

In [ ]:
render_code("4way_madd/madd.c", show=["//START","//END"])

Let's run it without the above loop code to figure the baseline memory accesses without running the loop on a Jetson nano.

#### Run without the 4-way matrix add loop code.

In [ ]:
# Run it "without" the above code.
! ssh htseng@nano "lscpu; cd courses/CS203/demo/memory/4way_madd/; make clean madd_nano; valgrind --tool=cachegrind ./madd_nano 16384 0 "

Too much detail! Let's use grep to narrow down the outputs.

In [ ]:
# Run it "without" the above code.
! ssh htseng@nano "cd courses/CS203/demo/memory/4way_madd/; valgrind --tool=cachegrind ./madd_nano 8192 0 >& nano_without_loop.perf; grep 'D   refs\|D1' nano_without_loop.perf"

Let's run it with the above loop code again and observe the changes in L1 cache misses/accesses

#### Run with the 4-way matrix add loop code.

In [ ]:
! ssh htseng@nano "cd courses/CS203/demo/memory/4way_madd/;valgrind --tool=cachegrind ./madd_nano 8192 8192 >& nano_with_loop.perf; grep 'D   refs\|D1' nano_with_loop.perf"

In [ ]:
# Let's do some math here
total_number_of_accesses_before_the_loop =  1527706
total_number_of_accesses_after_the_loop = 1,548192
total_number_of_accesses_in_the_loop = total_number_of_accesses_after_the_loop-total_number_of_accesses_before_the_loop
total_number_of_misses_before_the_loop = 8530
total_number_of_misses_after_the_loop = 29021
total_number_of_misses_in_the_loop = total_number_of_misses_after_the_loop-total_number_of_misses_before_the_loop
miss_rate_of_the_loop = total_number_of_misses_in_the_loop/total_number_of_accesses_in_the_loop

print(f"access in the loop: %d misses in the loop %d miss_rate %lf" % (total_number_of_accesses_in_the_loop, total_number_of_misses_in_the_loop, miss_rate_of_the_loop))

In [ ]:
! cd 4way_madd; make madd_dump; cd ..; 
!echo "element,address"> addresses_madd.csv; 
!./4way_madd/madd_dump 8192 8192 2>> addresses_madd.csv
! head -n 101 addresses_madd.csv > addresses_digest.csv
df = pd.read_csv("addresses_digest.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
C = 32768
B = 64
A = 4
offset_bits = int(math.log2(B))
S = int(C/(B*A))
index_bits = int(math.log2(S))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits)%S))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

### AMD RyZen 5 5500 -- 8-way L1, 64B-blocked, 32KB cache

Let's run it without the above loop code to figure the baseline memory accesses without running the loop on a Jetson nano.

Let's again dump, parse and simulation the address sequence.

In [ ]:
! cd 4way_madd; make madd_dump; cd ..; 
! echo "element,address"> addresses_madd.csv; 
! cs203 job run "./4way_madd/madd_dump 8192 8192 2>> addresses_madd.csv"
! head -n 101 addresses_madd.csv > addresses_digest.csv
df = pd.read_csv("addresses_digest.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
C = 49152
B = 64
A = 12
offset_bits = int(math.log2(B))
S = int(C/(B*A))
index_bits = int(math.log2(S))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % S))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

#### Run without the 4-way matrix add loop code.

In [ ]:
# Run it "without" the above code.
! cd ~/courses/CS203/demo/memory/4way_madd/
! cs203 job run "lscpu; rm ./4way_madd/madd_intel; make -C 4way_madd madd_intel; valgrind --tool=cachegrind ./4way_madd/madd_intel 16384 0 2>> amd_without_loop.perf"
! grep 'D   refs\|D1' amd_without_loop.perf

Let's run it with the above loop code again and observe the changes in L1 cache misses/accesses

#### Run with the 4-way matrix add loop code.

In [ ]:
# Run it "with" the above code.
! cs203 job run "valgrind --tool=cachegrind ./4way_madd/madd_intel 16384 16384  2>> amd_with_loop.perf"
! grep 'D   refs\|D1' amd_with_loop.perf

In [ ]:
# Let's do some math here
total_number_of_accesses_before_the_loop =2289801
total_number_of_accesses_after_the_loop = 2330796
total_number_of_accesses_in_the_loop = total_number_of_accesses_after_the_loop-total_number_of_accesses_before_the_loop
total_number_of_misses_before_the_loop = 13544
total_number_of_misses_after_the_loop = 23789
total_number_of_misses_in_the_loop = total_number_of_misses_after_the_loop-total_number_of_misses_before_the_loop
miss_rate_of_the_loop = total_number_of_misses_in_the_loop/total_number_of_accesses_in_the_loop

print(f"access in the loop: %d misses in the loop %d miss_rate %lf" % (total_number_of_accesses_in_the_loop, total_number_of_misses_in_the_loop, miss_rate_of_the_loop))

### Intel Core i7 12700K -- 12-way L1, 64B-blocked, 48KB cache

Let's run it without the above loop code to figure the baseline memory accesses without running the loop on a Jetson nano.

Let's again dump, parse and simulation the address sequence.

In [ ]:
! cd 4way_madd; make madd_dump; cd ..; 
!echo "element,address"> addresses_madd.csv; 
!./4way_madd/madd_dump 8192 8192 2>> addresses_madd.csv
! head -n 101 addresses_madd.csv > addresses_digest.csv
df = pd.read_csv("addresses_digest.csv",skipfooter=1,engine='python')
df["address"] = df["address"].str.replace('0x','')
df["address"]=df[["address"]].apply(lambda x: x.astype(str).map(lambda x: int(x, base=16)))
# only show the first N addresses 
#N = 32
#df2 = df2.iloc[:N]
C = 49152
B = 64
A = 12
offset_bits = int(math.log2(B))
S = int(C/(B*A))
index_bits = int(math.log2(S))
df["tag"]=(df["address"].apply(lambda x: x >> (offset_bits+index_bits)))
df["tag"] = df["tag"].apply(lambda x: hex(x))
df["index"] = df["address"].apply(lambda x: hex((x>>offset_bits) % S))
df["address"] = df["address"].apply(lambda x: hex(x))
display_df_mono(df)

#### Run without the 4-way matrix add loop code.

In [ ]:
# Run it "without" the above code.
! lscpu; cd ~/courses/CS203/demo/memory/4way_madd/; rm madd_intel; make madd_intel; valgrind --tool=cachegrind ./madd_intel 16384 0 >& intel_without_loop.perf; grep 'D   refs\|D1' intel_without_loop.perf

Let's run it with the above loop code again and observe the changes in L1 cache misses/accesses

#### Run with the 4-way matrix add loop code.

In [ ]:
# Run it "with" the above code.
! cd ~/courses/CS203/demo/memory/4way_madd/; valgrind --tool=cachegrind ./madd_intel 16384 16384  >& intel_with_loop.perf; grep 'D   refs\|D1' intel_with_loop.perf

In [ ]:
# Let's do some math here
total_number_of_accesses_before_the_loop =2285911
total_number_of_accesses_after_the_loop = 2326877
total_number_of_accesses_in_the_loop = total_number_of_accesses_after_the_loop-total_number_of_accesses_before_the_loop
total_number_of_misses_before_the_loop = 13447
total_number_of_misses_after_the_loop = 23694
total_number_of_misses_in_the_loop = total_number_of_misses_after_the_loop-total_number_of_misses_before_the_loop
miss_rate_of_the_loop = total_number_of_misses_in_the_loop/total_number_of_accesses_in_the_loop

print(f"access in the loop: %d misses in the loop %d miss_rate %lf" % (total_number_of_accesses_in_the_loop, total_number_of_misses_in_the_loop, miss_rate_of_the_loop))

## Prefetch

x86 provide prefetch instructions. As a programmer, you may insert ```_mm_prefetch``` in x86 programs to perform software prefetch for your code. The gcc compiler also has a flag ```-fprefetch-loop-arrays``` to automatically insert software prefetch instructions.

### Using prefetch in matrix transpose code

The following example is a highly optimized matrix transpose code. In the example, we try to prefetch the next row.

In [ ]:
render_code("./prefetch/transpose.cpp", lang="c++", show=["//START", "//END"])

Now, let's take a look of what's happening!

In [ ]:
! cd prefetch; make clean; make
# ! echo "Without prefetch -- the baseline"; ssh htseng@celebi "lscpu | grep Model; cd courses/CS203/demo/memory/prefetch/; ./transpose"
! echo "Without prefetch -- the baseline"
! lscpu | grep Model
! ./prefetch/transpose

In [ ]:
# ! echo "With prefetch"; ssh htseng@celebi " cd courses/CS203/demo/memory/prefetch/; ./transpose_prefetch"
! echo "With prefetch"
! ./prefetch/transpose_prefetch

Let's try a different machine now.

In [ ]:
#! cd prefetch; make clean; make; lscpu | grep Model
#! echo "Without prefetch -- the baseline"; cd prefetch/; ./transpose
#! echo "With prefetch"; cd prefetch/; ./transpose_prefetch
! cs203 job run "make -C ./prefetch clean; make -C ./prefetch ; lscpu | grep Model"
! echo "Without prefetch -- the baseline"; cs203 job run  "./prefetch/transpose"
! echo "With prefetch"; cs203 job run  "./prefetch/transpose_prefetch"


-- It doesn't work always!